<a href="https://colab.research.google.com/github/shokoufeh-monjezi/flowtron/blob/master/GNMT_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi


The below code check whether a Tensor core GPU is present.



In [ ]:

from tensorflow.python.client import device_lib

def check_tensor_core_gpu_present():
    local_device_protos = device_lib.list_local_devices()
    for line in local_device_protos:
        if "compute capability" in str(line):
            compute_capability = float(line.physical_device_desc.split("compute capability: ")[-1])
            if compute_capability>=7.0:
                return True
    
print("Tensor Core GPU Present:", check_tensor_core_gpu_present())
tensor_core_gpu = check_tensor_core_gpu_present()

In [1]:
!git clone https://github.com/NVIDIA/DeepLearningExamples

Cloning into 'DeepLearningExamples'...
remote: Enumerating objects: 26402, done.
remote: Counting objects: 100% (2053/2053), done.
remote: Compressing objects: 100% (1144/1144), done.
remote: Total 26402 (delta 891), reused 1904 (delta 870), pack-reused 24349
Receiving objects: 100% (26402/26402), 77.00 MiB | 21.83 MiB/s, done.
Resolving deltas: 100% (19399/19399), done.


In [2]:
import os

WORKSPACE_DIR='/content/DeepLearningExamples/TensorFlow/Translation/GNMT'
os.chdir(WORKSPACE_DIR)
print (os.getcwd())

/content/DeepLearningExamples/TensorFlow/Translation/GNMT


In [3]:
!ls


attention_wrapper.py	estimator.py	 model.py   requirements.txt
beam_search_decoder.py	examples	 nmt.py     scripts
benchmark_hooks.py	gnmt_model.py	 NOTICE     utils
block_lstm.py		img		 qa	    variable_mgr
Dockerfile		model_helper.py  README.md


### Download and preprocess the dataset.

The GNMT v2 model was trained on the WMT16 English-German dataset and newstest2014 is used as a testing dataset. We use scripts/wmt16_en_de.sh download script which automatically downloads and preprocesses the training and test datasets. By default, data is downloaded to the data directory.Data will be downloaded to the data directory (on the host). The data directory is mounted to the /workspace/gnmt/data location in the Docker container. 

In [4]:
pip install git+https://github.com/NVIDIA/dllogger.git

  Cloning https://github.com/NVIDIA/dllogger.git to /tmp/pip-req-build-mkyvmrgx
  Running command git clone -q https://github.com/NVIDIA/dllogger.git /tmp/pip-req-build-mkyvmrgx
  Created wheel for DLLogger: filename=DLLogger-0.1.0-py3-none-any.whl size=5656 sha256=a46ecff807d65f3e475c7ffd49b569bd3ccd85c9999b0f3be65302bcbd60eefa
  Stored in directory: /tmp/pip-ephem-wheel-cache-jod60qin/wheels/db/ba/1b/87515aba93adffc7caccc21c0e93f80b70a857188790ce0436
Successfully built DLLogger


In [5]:
!bash scripts/wmt16_en_de.sh

Writing to data/wmt16_de_en. To change this, set the OUTPUT_DIR environment variable.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  188M  100  188M    0     0  4299k      0  0:00:44  0:00:44 --:--:-- 4502k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  875M  100  875M    0     0  4005k      0  0:03:43  0:03:43 --:--:-- 4423k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 71.6M  100 71.6M    0     0  7876k      0  0:00:09  0:00:09 --:--:-- 10.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.7M  100 21.7M    0     0  5082k      0

## Start Training.

All results and logs are saved to the results directory (on the host) or to the /workspace/gnmt/results directory (in the container). The training script saves the checkpoint after every training epoch and after every 2000 training steps within each epoch. You can modify the results directory using the --output_dir argument.

To launch mixed precision training on 1 GPU, run:
```
python nmt.py --output_dir=results --batch_size=128 --learning_rate=5e-4 --amp
```
To launch mixed precision training on 8 GPUs, run:
```
python nmt.py --output_dir=results --batch_size=1024 --num_gpus=8 --learning_rate=2e-3 --amp
```
To launch FP32 (TF32 on NVIDIA Ampere GPUs) training on 1 GPU, run:
```
python nmt.py --output_dir=results --batch_size=128 --learning_rate=5e-4
```
To launch FP32 (TF32 on NVIDIA Ampere GPUs) training on 8 GPUs, run:
```
python nmt.py --output_dir=results --batch_size=1024 --num_gpus=8 --learning_rate=2e-3
```

In [8]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.6.0


In [9]:
!pip install tensorflow==1.13.2

     |████████████████████████████████| 92.7 MB 11 kB/s 
     |████████████████████████████████| 3.2 MB 54.6 MB/s 
     |████████████████████████████████| 367 kB 49.1 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow 1.13.2 which is incompatible.


In [10]:
!python nmt.py --output_dir=results --batch_size=128 --learning_rate=5e-4 --amp


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

## Start evaluation.

The training process automatically runs evaluation and outputs the BLEU score after each training epoch. Additionally, after the training is done, you can manually run inference on test dataset with the checkpoint saved during the training.

To launch mixed precision inference on 1 GPU, run:
```
python nmt.py --output_dir=results --infer_batch_size=128 --mode=infer --amp
```
To launch FP32 (TF32 on NVIDIA Ampere GPUs) inference on 1 GPU, run:
```
python nmt.py --output_dir=results --infer_batch_size=128 --mode=infer
```

In [ ]:
!python nmt.py --output_dir=results --infer_batch_size=128 --mode=infer --amp


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

## Start translation.

After the training is done, you can translate custom sentences with the checkpoint saved during the training.
```
echo "The quick brown fox jumps over the lazy dog" >file.txt python nmt.py --output_dir=results --mode=translate --translate-file=file.txt cat file.txt.trans
```

In [ ]:
!echo "The quick brown fox jumps over the lazy dog" >file.txt python nmt.py --output_dir=results --mode=translate --translate-file=file.txt cat file.txt.trans

## Other command options
To see the full list of available options and their descriptions, use the -h or --help command line option, for example:



In [ ]:
!python nmt.py --help


2021-06-11 04:01:29.489201: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



usage: nmt.py [-h] [--num_units NUM_UNITS] [--num_layers NUM_LAYERS]
              [--num_encoder_layers NUM_ENCODER_LAYERS]
              [--num_decoder_layers NUM_DECODER_LAYERS]
              [--encoder_type ENCODER_TYPE] [--residual [RESIDUAL]]
              [--time_major [TIME_MAJOR]]
              [--num_embeddings_partitions NUM_EMBEDDINGS_PARTITIONS]
              [--attention ATTENTION]
              [--attention_architecture ATTENTION_ARCHITECTURE]
              [--output_a